In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# ==========================================
# 1. GLOBAL CONFIGURATION (The "Common Keys")
# ==========================================
# We use these exact names across all 4 datasets to ensure they join in Power BI.
ZONES = ['Hinjewadi', 'Kothrud', 'Swargate', 'Koregaon Park', 'Hadapsar', 'Viman Nagar']
START_DATE = datetime(2025, 1, 1)
DAYS = 30  # Generating 1 month of data

# Helper to generate hourly timestamps
def get_hourly_timeline(start, days):
    timeline = []
    current = start
    end = start + timedelta(days=days)
    while current < end:
        timeline.append(current)
        current += timedelta(hours=1)
    return timeline

timeline = get_hourly_timeline(START_DATE, DAYS)

# ==========================================
# DATASET 1: TRAFFIC FLOW (Mobility)
# ==========================================
print("Generating 'pune_traffic_flow.csv'...")
traffic_data = []

for ts in timeline:
    for zone in ZONES:
        # Pune Context: Hinjewadi & Hadapsar are IT/Industrial (High peak traffic)
        hour = ts.hour
        is_weekend = ts.weekday() >= 5
        
        # Peak Hours: 9-11 AM and 6-9 PM
        is_peak = (9 <= hour <= 11) or (18 <= hour <= 21)
        
        if zone in ['Hinjewadi', 'Hadapsar'] and is_peak and not is_weekend:
            congestion = np.random.randint(85, 100) # Gridlock
            avg_speed = np.random.randint(5, 15)
        elif zone == 'Koregaon Park' and hour > 20 and is_weekend:
            congestion = np.random.randint(70, 90) # Nightlife traffic
            avg_speed = np.random.randint(10, 20)
        else:
            congestion = np.random.randint(10, 50)
            avg_speed = np.random.randint(30, 60)

        traffic_data.append({
            'Date': ts.date(),
            'Hour': ts.hour,
            'Zone_Name': zone,
            'Junction_ID': f"{zone[:3].upper()}_J01",
            'Vehicle_Volume': int(congestion * 15), # Proxy for volume
            'Avg_Speed_Kmph': avg_speed,
            'Congestion_Index': congestion # 0-100 scale
        })

df_traffic = pd.DataFrame(traffic_data)
df_traffic.to_csv('pune_traffic_flow.csv', index=False)

# ==========================================
# DATASET 2: UTILITIES (Energy Consumption)
# ==========================================
print("Generating 'pune_energy_consumption.csv'...")
energy_data = []

for ts in timeline:
    for zone in ZONES:
        # Context: Commercial vs Residential patterns
        hour = ts.hour
        
        # Base Load
        if zone in ['Hinjewadi', 'Viman Nagar']: # Commercial Zones
            load = np.random.normal(5000, 500) if (9 <= hour <= 19) else np.random.normal(1500, 200)
        else: # Residential Zones
            load = np.random.normal(3000, 300) if (18 <= hour <= 23) else np.random.normal(1000, 100)
            
        # Voltage Fluctuation logic (Pune outskirts often face dips)
        voltage = 240 + np.random.normal(0, 5)
        if zone == 'Hadapsar' and np.random.random() > 0.95: 
            voltage = 190 # Low voltage event
            
        energy_data.append({
            'Date': ts.date(),
            'Hour': ts.hour,
            'Zone_Name': zone,
            'Feeder_ID': f"MSEDCL_{zone[:3]}_F1",
            'Energy_Consumption_kWh': round(load, 2),
            'Grid_Voltage': round(voltage, 2),
            'Power_Cut_Flag': 1 if voltage < 200 else 0
        })

df_energy = pd.DataFrame(energy_data)
df_energy.to_csv('pune_energy_consumption.csv', index=False)

# ==========================================
# DATASET 3: WASTE MANAGEMENT (Smart Bins)
# ==========================================
print("Generating 'pune_waste_management.csv'...")
waste_data = []
# Waste data is usually daily or per collection trip, not hourly.
# But for Power BI connectivity, we will log it at 8 AM (Collection time) everyday.

waste_dates = pd.date_range(start=START_DATE, periods=DAYS, freq='D')

for date in waste_dates:
    for zone in ZONES:
        # Context: Kothrud generates more residential wet waste
        total_bins = 50
        
        if zone == 'Kothrud':
            avg_fill = np.random.randint(70, 95)
        else:
            avg_fill = np.random.randint(40, 80)
            
        # Segregation efficiency (Swachh Bharat metric)
        segregation_score = np.random.randint(60, 95)
        
        waste_data.append({
            'Date': date.date(),
            'Zone_Name': zone,
            'Total_Waste_Collected_Kg': np.random.randint(500, 2000),
            'Avg_Bin_Fill_Level_Percent': avg_fill,
            'Segregation_Efficiency_Percent': segregation_score,
            'Missed_Pickups': np.random.randint(0, 5)
        })

df_waste = pd.DataFrame(waste_data)
df_waste.to_csv('pune_waste_management.csv', index=False)

# ==========================================
# DATASET 4: CITIZEN GRIEVANCES (PMC Care)
# ==========================================
print("Generating 'pune_citizen_grievances.csv'...")
grievance_data = []

categories = {
    'Traffic': ['Traffic Signal Not Working', 'Potholes', 'Illegal Parking'],
    'Energy': ['Frequent Power Cut', 'Voltage Fluctuation', 'Meter Fault'],
    'Waste': ['Garbage Dumped on Road', 'Bin Overflowing', 'Dead Animal'],
    'Water': ['No Water Supply', 'Contaminated Water']
}

# Generate 1000 random complaints
for _ in range(1000):
    date = START_DATE + timedelta(days=random.randint(0, DAYS-1))
    zone = random.choice(ZONES)
    
    # Correlate complaints with other data (Advanced realism)
    # If it's Hinjewadi, bias towards Traffic complaints
    if zone == 'Hinjewadi':
        dept = 'Traffic' if random.random() > 0.4 else random.choice(list(categories.keys()))
    else:
        dept = random.choice(list(categories.keys()))
        
    issue = random.choice(categories[dept])
    
    grievance_data.append({
        'Date': date.date(),
        'Zone_Name': zone,
        'Ticket_ID': f"PMC{random.randint(10000,99999)}",
        'Department': dept,
        'Issue_Type': issue,
        'Status': np.random.choice(['Resolved', 'Open', 'In Progress'], p=[0.6, 0.3, 0.1]),
        'SLA_Days': np.random.randint(1, 5)
    })

df_grievances = pd.DataFrame(grievance_data)
df_grievances.to_csv('pune_citizen_grievances.csv', index=False)

print("\nSUCCESS! 4 Power BI-ready CSV files created.")

Generating 'pune_traffic_flow.csv'...
Generating 'pune_energy_consumption.csv'...
Generating 'pune_waste_management.csv'...
Generating 'pune_citizen_grievances.csv'...

SUCCESS! 4 Power BI-ready CSV files created.


In [2]:
df1=pd.read_csv('pune_traffic_flow.csv')
df2=pd.read_csv('pune_energy_consumption.csv')
df3=pd.read_csv('pune_waste_management.csv')    
df4=pd.read_csv('pune_citizen_grievances.csv')

In [3]:
df1

,Date,Hour,Zone_Name,Junction_ID,Vehicle_Volume,Avg_Speed_Kmph,Congestion_Index
0,2025-01-01,0,Hinjewadi,HIN_J01,630,42,42
1,2025-01-01,0,Kothrud,KOT_J01,585,52,39
2,2025-01-01,0,Swargate,SWA_J01,435,31,29
3,2025-01-01,0,Koregaon Park,KOR_J01,630,53,42
4,2025-01-01,0,Hadapsar,HAD_J01,630,50,42
...,...,...,...,...,...,...,...
4315,2025-01-30,23,Kothrud,KOT_J01,225,44,15
4316,2025-01-30,23,Swargate,SWA_J01,225,36,15
4317,2025-01-30,23,Koregaon Park,KOR_J01,735,48,49
4318,2025-01-30,23,Hadapsar,HAD_J01,420,38,28


In [4]:
df2

,Date,Hour,Zone_Name,Feeder_ID,Energy_Consumption_kWh,Grid_Voltage,Power_Cut_Flag
0,2025-01-01,0,Hinjewadi,MSEDCL_Hin_F1,1393.85,236.78,0
1,2025-01-01,0,Kothrud,MSEDCL_Kot_F1,724.46,236.23,0
2,2025-01-01,0,Swargate,MSEDCL_Swa_F1,1190.06,245.03,0
3,2025-01-01,0,Koregaon Park,MSEDCL_Kor_F1,880.61,243.86,0
4,2025-01-01,0,Hadapsar,MSEDCL_Had_F1,1122.99,232.41,0
...,...,...,...,...,...,...,...
4315,2025-01-30,23,Kothrud,MSEDCL_Kot_F1,3361.75,237.68,0
4316,2025-01-30,23,Swargate,MSEDCL_Swa_F1,2989.40,238.92,0
4317,2025-01-30,23,Koregaon Park,MSEDCL_Kor_F1,3046.34,239.31,0
4318,2025-01-30,23,Hadapsar,MSEDCL_Had_F1,2325.48,242.31,0


In [5]:
df3

,Date,Zone_Name,Total_Waste_Collected_Kg,Avg_Bin_Fill_Level_Percent,Segregation_Efficiency_Percent,Missed_Pickups
0,2025-01-01,Hinjewadi,1078,59,90,0
1,2025-01-01,Kothrud,958,73,82,4
2,2025-01-01,Swargate,1593,60,61,4
3,2025-01-01,Koregaon Park,1878,40,83,4
4,2025-01-01,Hadapsar,1517,58,60,4
...,...,...,...,...,...,...
175,2025-01-30,Kothrud,1379,80,80,0
176,2025-01-30,Swargate,1771,63,84,3
177,2025-01-30,Koregaon Park,1325,51,75,2
178,2025-01-30,Hadapsar,1346,78,72,1


In [6]:
df4

,Date,Zone_Name,Ticket_ID,Department,Issue_Type,Status,SLA_Days
0,2025-01-11,Viman Nagar,PMC25657,Traffic,Illegal Parking,Open,4
1,2025-01-26,Viman Nagar,PMC71006,Water,Contaminated Water,Open,1
2,2025-01-14,Swargate,PMC73440,Waste,Dead Animal,In Progress,3
3,2025-01-15,Viman Nagar,PMC16031,Waste,Dead Animal,Open,3
4,2025-01-14,Koregaon Park,PMC54975,Water,No Water Supply,Resolved,2
...,...,...,...,...,...,...,...
995,2025-01-20,Hadapsar,PMC50092,Waste,Garbage Dumped on Road,In Progress,2
996,2025-01-02,Swargate,PMC49959,Energy,Frequent Power Cut,Resolved,2
997,2025-01-11,Koregaon Park,PMC47552,Traffic,Traffic Signal Not Working,Open,2
998,2025-01-19,Hinjewadi,PMC36041,Traffic,Illegal Parking,In Progress,2
